<a href="https://colab.research.google.com/github/CarlosRochaCx/MNA/blob/main/MNA_IAyAA_semana_8_Actividad_Sistemas_Recomendacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Maestría en Inteligencia Artificial Aplicada**
### **Curso: Inteligencia Artificial y Aprendizaje Automático**
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad de la Semana 8**
#### **Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Esta Actividad deberás resolverla de manera individual.**

**Nombre y matrícula:**

*   
*   


In [26]:
# Incluye aquí todas las librerías y paquetes que requieras.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.decomposition import TruncatedSVD

### **Liga de Datos de la UCI "Restaurant & consumer data".**
### **Descarga el archivo "restaurant+consumer+data.zip" y de ahí necesitaremos los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data



In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MNA - Maestría en Inteligencia Artificial Aplicada/Inteligencia Artificial y Aprendizaje Automático/Tareas/Actividad Semana 8/rating_final.csv", header='infer', sep=",")
data2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MNA - Maestría en Inteligencia Artificial Aplicada/Inteligencia Artificial y Aprendizaje Automático/Tareas/Actividad Semana 8/geoplaces2.csv", header='infer',  encoding='latin-1')

print(data1.shape, data2.shape)


(1161, 5) (130, 21)


**NOTA: Recuerda que cada vez que se te pida incluir líneas de código, podrás incluir 1 o más líneas, pero siempre en la sección donde se te indica hacerlo.**

## **Ejercicio - 1**

### **Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

++++++++ Inicia la sección de agregar texto: +++++++++++

El argumento ```encoding='latin-1'``` se usa para leer correctamente archivos que tienen letras especiales, como acentos o la letra ñ. Esto es común en textos en español o en otros idiomas europeos. Si no se usa esta opción, pueden aparecer errores o símbolos raros al cargar los datos. En este caso, el archivo ```geoplaces2.csv``` contiene nombres de lugares en México, los cuales contienen este tipo de letras. Por eso se indica ```latin-1```, para asegurarse de que el texto se vea bien.

++++++++ Termina la sección de agregar texto. +++++++++++

In [29]:
# Del primer archivo obtenemos una matriz con 3 evaluaciones de los restaurantes:
# general, comida y servicio.
# Las evaluaciones pueden ser 0, 1 o 2, siendo 0 la menor calificación y 2 la
# mayor calificación:

data1.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [30]:
# Del segundo archivo obtenemos información diversa de cada restaurante:

data2.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [31]:
# De data1 no requeriremos "rating" y de data2 solo necesitamos "placeID" y "name":
# Definimos la lista y matrices con los factores que necesitamos:

lista_data1 = ['userID','placeID','food_rating','service_rating']
lista_data2 = ['placeID','name']

data1a = data1[lista_data1]
data2a = data2[lista_data2]

## **Ejercicio - 2**

In [32]:
# Define el DataFrame que conjunte la información de las dos DataFrame data1a
# y data2a en uno solo, a través de la columna común "placeID" y que
# llamaremos "df_combinado":



# ************* Inlcuye aquí tu código:*****************************


df_combinado = pd.merge(data1a, data2a, on='placeID')


# *********** Aquí termina la sección de agregar código *************


# Despleguemos la dimensión y los primeros renglones de este DataFrame:

print(df_combinado.shape)
df_combinado.head()

(1161, 5)


,userID,placeID,food_rating,service_rating,name
0,U1077,135085,2,2,Tortas Locas Hipocampo
1,U1077,135038,2,1,Restaurant la Chalita
2,U1077,132825,2,2,puesto de tacos
3,U1077,135060,2,2,Restaurante Marisco Sam
4,U1068,135104,1,2,vips


## **Ejercicio - 3**

In [33]:
#    Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación de la comida (food_rating). La llamaremos "UtMx_food".


# ************* Inlcuye aquí tu código:*****************************


UtMx_food = pd.pivot_table(
    df_combinado,
    values='food_rating',
    index='name',
    columns='userID',
    fill_value=0)

# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz de Utilidad sobre la evaluación de la comida:')
print('(restaurantes, usuarios) =', (UtMx_food.shape))
UtMx_food.head()

Dimensión de la matriz de Utilidad sobre la evaluación de la comida:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arrachela Grill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cabana Huasteca,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Cafe Chaires,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cafeteria cenidet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **Ejercicio - 4**

### **En la Factorización SVD la cantidad de valores singulares será menor o igual al menor valor de los renglones o columnas de la matriz de utilidad UtMx.**

### **La factorización SVD de una matriz $A$ tiene la forma:**

$A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

In [34]:
# a. Aplica la transformación SVD para obtener la matriz de las variables
#    latentes de los restaurantes, en relación a la manera en que los usuarios
#    evaluaron la comida. Deberás obtener la factorización SVD con el máximo número
#    de componentes y por lo tanto el máximo de valores singulares posibles de acuerdo
#    a este criterio. Así, de acuerdo al resultado obtenido en el Ejercicio-3, asigna
#    a la variable "nc_food" el número entero adecuado para llegar a obtener el
#    resultado pedido.


# ************* Inlcuye aquí tu código:**********************************************


nc_food = min(UtMx_food.shape) # número de componentes


# *********** Aquí termina la sección de agregar código *****************************

print('Total de valores singulares basados en evaluación de la comida:', nc_food)

Total de valores singulares basados en evaluación de la comida: 129


In [35]:
# b. Lleva a cabo la factorización SVD truncada en la matriz de utilidad
#    obtenida previamente y con respecto al número de componentes dada
#    por la variable nc_food.


# ************* Inlcuye aquí tu código:********************************


SVD_food = TruncatedSVD(n_components = nc_food)
resultant_matrix_food = SVD_food.fit_transform(UtMx_food)

# *********** Aquí termina la sección de agregar código ***************


# Con la factorización SVD truncada obtenida, determinemos la cantidad
# de componentes que explican un 90% de la variabilidad acumulada de
# cada componente:

for j in range(nc_food):
  if SVD_food.explained_variance_ratio_[0:j].sum() > 0.90:
    break

# Usaremos esta cantidad de componentes para las recomendaciones basadas
# en la calificación de la comida:
N_food = j-1


print('Valor de truncamiento al 90% de dicha variabilidad:', N_food)

Valor de truncamiento al 90% de dicha variabilidad: 51


In [36]:
# Usando la métrica de correlación de Pearson, obtenemos ahora las 10 mejores
# recomendaciones de este modelo no supervisado con base a la información de
# alguien que desea obtener las similitudes con el "Restaurante Pueblo Bonito"
# y con la cota de truncamiento obtenido en el ejercicio anterior.
# Para ello obtenemos las 10 mejores correlaciones positivas a continuación.

# Factorización SVD:
SVD_food = TruncatedSVD(n_components = N_food)
resultant_matrix_food = SVD_food.fit_transform(UtMx_food)

# Matriz de correlación de Pearson:
corr_mat_food = np.corrcoef(resultant_matrix_food)

# Restaurante de referencia:
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_food.T.columns  # nombres de restaurantes
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_food[idx_rest] # Vector de Correlación del RinconHuasteco

# Buscando las correlaciones positivas:
idx = (corr_rest>0)
mejores_sim_food = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_food.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_food))

print('Algunos de los resultados encontrados:')
mejores_sim_food[0:5]

Total de similaridades positivas encontradas: 66
Algunos de los resultados encontrados:


[(np.float64(0.048069180163315885), 'Cabana Huasteca'),
 (np.float64(0.018216125942904598), 'Cafe Chaires'),
 (np.float64(0.007387836329854431), 'Cafeteria cenidet'),
 (np.float64(0.2392654552514337), 'Cafeteria y Restaurant El Pacifico'),
 (np.float64(0.027417809195348013), 'Cenaduria El RincÃ³n de Tlaquepaque')]

## **Ejercicio - 5**

In [37]:
# Ordenar la lista de recomendaciones "mejores_sim_food" encontrada en el paso
# anterior de manera descendente. Llamaremos "mejores_sim_food_ordenadas" a
# dicha lista.

# ************* Inlcuye aquí tu código:*****************************

mejores_sim_food_ordenadas = sorted(mejores_sim_food, reverse=True)

# *********** Aquí termina la sección de agregar código *************


# Desplegamos las 10 mejores similitudes encontradas de manera descendente:
print('Similitudes con base a la evaluación de la comida con mayores valores de correlación:')
for k in range(1,11):
  print('%d> %s' % (k, mejores_sim_food_ordenadas[k]))

Similitudes con base a la evaluación de la comida con mayores valores de correlación:
1> (np.float64(0.7940370229672926), 'Restaurante la Estrella de Dima')
2> (np.float64(0.7047995846620417), 'pizza clasica')
3> (np.float64(0.6534888955077086), 'Restaurante Guerra')
4> (np.float64(0.5559197174198666), 'El Club')
5> (np.float64(0.3839796801415931), 'Pizzeria Julios')
6> (np.float64(0.30710296346546184), 'Hamburguesas saul')
7> (np.float64(0.3044981936082026), 'Restaurant Oriental Express')
8> (np.float64(0.29317821080261797), 'Restaurante El Cielo Potosino')
9> (np.float64(0.29295412429288165), 'la Cochinita Pibil Restaurante Yucateco')
10> (np.float64(0.2929092405238159), 'La Fontana Pizza Restaurante and Cafe')


## **Ejercicio - 6**

### **De manera análoga y usando ahora la evaluación de un restaurante con respecto a su servicio (service_rating), encuentra ahora los diez restaurantes con mayor similaridad al mismo restaurante anterior llamado "Restaurante Pueblo Bonito" y con respecto también a la misma métrica de similaridad de Pearson.**

### **Deberás usar la cantidad de componentes necesarias para que la varabilidad de la factorización SVD quede explicada en un 90%.**

### **a.  Para ello define primero la matriz de utilidad, UtMx_service, donde los renglones son los nombres de los restaurantes, las columnas los usuarios y las entradas las evaluaciones con respecto al servicio recibido en el restaurante.**





In [38]:
# a. Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación del servicio (service_rating). La llamaremos "UtMx_service".


# ************* Inlcuye aquí tu código:*****************************

UtMx_service = pd.pivot_table(
    df_combinado,
    values='service_rating',
    index='name',
    columns='userID',
    fill_value=0)

# *********** Aquí termina la sección de agregar código *************




print('Dimensión de la matriz de Utilidad sobre la evaluación del servicio:')
print('(restaurantes, usuarios) =', (UtMx_service.shape))
UtMx_service.head()

Dimensión de la matriz de Utilidad sobre la evaluación del servicio:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arrachela Grill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cabana Huasteca,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Cafe Chaires,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cafeteria cenidet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# b. Incluye el código necesario para obtener y desplegar los 10 restaurantes con
#    mayor similaridad de Pearson al "Restaurante Pueblo Bonito", y usando solo
#    el 90% de las componentes que explican la varianza de la matriz UtMx_service.
#    Desplegarlos en orden descendente, sin incliur el restaurante de referencia.

# ************* Inlcuye aquí tu código:*****************************

# Paso 1: Aplicar SVD con un número alto de componentes para captar varianza total
max_components = min(UtMx_service.shape)
svd_full = TruncatedSVD(n_components=max_components)
svd_full.fit(UtMx_service)

# Paso 2: Calcular varianza explicada acumulada
varianza_acum = np.cumsum(svd_full.explained_variance_ratio_)

# Paso 3: Número mínimo de componentes para alcanzar al menos 90% de varianza
nc_service = np.searchsorted(varianza_acum, 0.90) + 1  # +1 porque searchsorted da índice base 0

print(f"Número de componentes para explicar 90% de varianza: {nc_service}")

# Paso 4: Aplicar SVD con nc_service componentes
svd_service = TruncatedSVD(n_components=nc_service)
matriz_latente_service = svd_service.fit_transform(UtMx_service)

# Paso 5: Matriz de correlación de Pearson entre restaurantes en espacio latente
corr_mat_service = np.corrcoef(matriz_latente_service)

# Paso 6: Obtener índice del restaurante de referencia
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_service.index
idx_rest = list(nombres_rest).index(restaurante_de_referencia)

# Paso 7: Vector de correlación del restaurante de referencia
corr_rest = corr_mat_service[idx_rest]

# Paso 8: Crear lista de similitudes positivas, excluyendo el restaurante mismo
mejores_sim_service = [
    (corr, nombre) for corr, nombre in zip(corr_rest, nombres_rest)
    if corr > 0 and nombre != restaurante_de_referencia
]

# Paso 9: Ordenar lista en orden descendente
mejores_sim_service_ordenadas = sorted(mejores_sim_service, key=lambda x: x[0], reverse=True)

# Paso 10: Imprimir las 10 mejores recomendaciones
print("Top 10 restaurantes similares según servicio a", restaurante_de_referencia)
for sim, nombre in mejores_sim_service_ordenadas[:10]:
    print(f"{nombre}: {sim:.4f}")


# *********** Aquí termina la sección de agregar código *************

Número de componentes para explicar 90% de varianza: 52
Top 10 restaurantes similares según servicio a Restaurante Pueblo Bonito
pizza clasica: 0.6617
El Club: 0.5820
Restaurante la Estrella de Dima: 0.4785
Hamburguesas saul: 0.4760
Restaurante Guerra: 0.4288
Restaurant Orizatlan: 0.4134
la Cochinita Pibil Restaurante Yucateco: 0.3611
Restaurante El Cielo Potosino: 0.3492
La Fontana Pizza Restaurante and Cafe: 0.3135
puesto de tacos: 0.3000


## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular indica cuáles restaurantes encontraste con correlaciones altas tanto para la calificación de la comida, como del servicio.**

++++++++ Inicia la sección de agregar texto: +++++++++++

En esta actividad usamos la descomposición SVD para analizar las calificaciones de comida y servicio de varios restaurantes. Encontramos que algunos restaurantes, como “Restaurante Pueblo Bonito”, tienen similares evaluaciones en ambos aspectos con lugares como “pizza clasica” y “El Club”, que mostraron alta correlación en la valoración del servicio. Las correlaciones reflejan qué tan parecidos son los restaurantes según las opiniones de los usuarios. Esto ayuda a generar recomendaciones personalizadas basadas en gustos reales. En resumen, el método permite encontrar restaurantes parecidos en diferentes criterios de evaluación, mejorando la experiencia de recomendación.

++++++++ Termina la sección de agregar texto. +++++++++++

>> **Fin de la Actividad de Sistemas de Recomendación**